SUPPORT VECTOR MACHINES :

this is a type of supervised learning algo, 
they are powerful and versatile capable of handling complex decision boundaries making it suitable for a wide range of tasks 

linear classification : svm can separate data using a straight line in 2d or hyperplane 

non linear classificaion : using kernel tricks it can handle complex curved boundaries 

regression : svms can be adapted for regression tasks called SVR (support vector regressions)

novelty detection : svms can detect outliers and anomilies used in fraud detection or defect spotting 

they are good for small to medium datasets(hundreds to thousands) not millions, good for classification with a lot of features 

LINEAR SVM CLASSIFICATION :

SEE fig 5-1 :

left plot :
the dashed line doesnt even seperate the two classes properly 
pink and green lines seperate the classes correctly but they paas to some points very closely, they may not generalise well with the new data(overfitting issue)

right plot :
the solid black line is the decision boundary of the svm classifier and the dashed lines represent the margin lines 
svm maximizes the margin between the classes 
the suppors vectors(closest point in the decision boundary) lie on the dashboard margin lines 
this is called margin classification meaning the model tries to leave as much space between the two classess 
it doesnt just aim to seperate the classes but also finds the boundary that is furthest from any training point on either side 

if we add more training instances far from the margin the model wont change much as svm cares only about the closest points from each class 
the decision boundary is mathematically defined by few data points that lie on the margin, these are the most important instances for the model therefore called the support vectors these points are circled in fig 5-1 right plot 
all other points can be removed and the svm decision boundary wont change 

svms are highly sensitive to the scale of input features, if one feature has much larger range than other it can dominate the distance calc and that affects the decision boundary, seef fig 5-2

left plot :
x1 has larger range so the distance between the points is determined by differences in x1, resulting a near horizontal decision boundary, which results to a skewed street that doesnt reflect the true relationship betweeen the features 

right plot :
features are now standardized the decision boundaries are now diagonal which seperates the two classes better


see fig 5-3 :

hard margin classification :

svm strictly seperates the classes with a margin street this works only well when the data is perfectly linearly seperable, see fig 5-3 left plot :
an outlier is added this makes it impossible to draw a decision margin that seperates the two classes 

see fig 5-3 right plot :
in this example a hardl line is still attempted despite the outlier and the svm distorts the decision boundart to include that outlier 
the margin shrinks and generalisation will suffer, it wont perform well on unseen data

soft margin classification :
to avoid the above issue we need to use a more flexible model which will keep the street or margin as large as possible and limit the margin violations that means some will be misclassified but the majority will be inside the margin 


regularisation in svm :

we can use the svc class C param it controls the balance between the margin size and classification accuracy, see fig 5-4:

left plot: (c=1, soft margin)
larger margin(wider street), the svm allows more margin violations 
many support vectors are inside the margin(those with gray circles)
model is more prominent to errors less likely to overfit but it may fail to capture the real boundary

right plot (c=100, harder margin)
narrrower margin, model is trying to minimize the classification errors even if the margin shrinks
fewer margin violations are allowed (closer to a hard margin)
it may overfit but in this example it actually generalises better than the left model 

if your svm model is overfitting you can try regularize it by reducing the C

lets train a linear svm classifier now :

In [6]:
from sklearn.datasets import load_iris
iris  = load_iris(as_frame=True) #as_frame returns it as pandas dataframe

x = iris.data[["petal length (cm)", "petal width (cm)"]].values # a numpy array of two selected features

y = (iris.target == 2) #iris virginica
# a boolean label

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC

svm_clf = make_pipeline(StandardScaler(), LinearSVC(C=1, random_state=42)) #c=1 soft margin

svm_clf.fit(x,y)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('linearsvc', LinearSVC(C=1, random_state=42))])

this pipeline fits the scalar and svm model on the training data 
the resulting decision boundary corresponds to left plot wider margin more violations 

lets make predictions now :

In [7]:
x_new = [[5.5, 1.7], [5.0, 1.5]]
svm_clf.predict(x_new)

array([ True, False])

In [8]:
svm_clf.decision_function(x_new)

array([ 0.66163816, -0.22035761])

decision_function() returns the signed distance between the decision boundary and point 
positive  = classified as true 
negative = classified as false (on the other side of the margin)


LinearSVC is optimized for speed and scalability,  it does not support predict_proba(estimated class probab) and predict_log_proba

instead we can use SCV(probability = True) to use both the methods 
working:
trains the svm model 
performs the 5 fold cross validations and get out-of-sample decision scores
fits a logistic reg on these scores, as a result output probab are caliberated using the logistic function 

it leads to slower training as it include some extra steps (CV and caliberation)

NON LINEAR SVM CLASSIFICATION :

sometimes the data is not linearlly seperable so we transform it by adding non linear features and it can become linear in a higher dimensional space 
see fig 5-5 :

left plot :
all green points are in the middle and all blue points are in the ends it is impossible to draw a vertical line that seperates them so this dataset is non seperable in 1d space

right plot :
new feature, x2 = x1^2
x axis is still x1 but now we have a second axis(y) x2 
this simple non linear feature lifts the middle points lower and the end points higher on the vertical axis, now we can draw a horizontal line seperating the classes 

lets see how to implement this :

In [9]:
from sklearn.datasets import make_moons
x, y = make_moons(n_samples=100, noise=0.15, random_state=42) #this generates a 2d dataset with two interleaving moons , these moons represent two diff classes 

from sklearn.preprocessing import PolynomialFeatures

polynomial_svm_clf = make_pipeline(
    PolynomialFeatures(degree=3), #adds non linear polynomial features
    StandardScaler(),
    LinearSVC(C=10, max_iter=10000, random_state=42)
)

polynomial_svm_clf.fit(x,y)

Pipeline(steps=[('polynomialfeatures', PolynomialFeatures(degree=3)),
                ('standardscaler', StandardScaler()),
                ('linearsvc',
                 LinearSVC(C=10, max_iter=10000, random_state=42))])

see fig 5-6:

blue square and green triangles represent the two classes 
the coloured background blue and green represent the decision regions 
the decision boundary curves smoothly despite using the LinearSVC the decision boundary is non linear 
all this because we used the polynomial transformation 
so by mapping the data to a higher dimensional polynomial feature space LinearSVM can act like nonlinear classifier 

POLYNOMIAL KERNEL :

adding polynomial features is simple and can work with almost all sort of machine learning algo not just svm, the low polynomial degree cannot work with complex dataset, and with a higher polynomial degree it creates a huge no of features making the model too slow

lets implement the polynomial kernel trick using the moons dataset

In [10]:
from sklearn.svm import SVC
polyk_kernel_svm_clf = make_pipeline(
    StandardScaler(),
    SVC(kernel="poly", degree=3, coef0=1, C=5)
)
polyk_kernel_svm_clf.fit(x,y)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc', SVC(C=5, coef0=1, kernel='poly'))])

kernel="poly" uses the polynomial kernel trick(instead of manually adding them)
the colourful decision boundary between crescents in fig 5-6 is the result of such non linear decision function
it avoids the combinational explosion of features and is mathematically equivalent to adding polynomial features 

see fig 5-7

left plot :
moderately complex decision boundary smooth and generalises well
right plot :
very complex boundary showing overfitting

degree :
higher degrees more curves
degreee = 3 (handles non linear data well)
degree = 10 (overfits - fits noise in training data)

coef0 :
controls how much influence higher degree polynomial have 
coef = 100 (right) causes the higher degree behavior to dominate worsening overfitting 

polynomial kernel can approxiamate very complex boundaries but :
too much complexity = the model memorizes the training data (high variance)
not enough = the model cant capture the underlying patterns (high bias)

the art is tuning by using grid search + cross validation to find the best combination of degree, coef0 and C

although tools like randomized search cv and gridsearch cv can search the best hyperparameters automatically, but understanding how each hyperparameter works is helpul :

1. reduces search space :
instead of blindly searching for the best pair of combinations one can easily eliminate unreasonable values (eg, degree>20 is rarely useful)
focus on combinations that are more likely to perform well

2. improves efficiency and speed :
auto tuning can be computationally expensive on large datasets, you will also interpret results better when you understand what the algorithm is adjusting

3. enhances debugging and fine tuning :
when performance drops or results seem off you can identify the cause and(maybe coef0 is high), you can intervene manually and make adjustments 

| Hyperparameter | Controls                        | Effect                                             |
| -------------- | ------------------------------- | -------------------------------------------------- |
| `degree`       | Polynomial degree               | Higher = more flexible, but more overfitting risk 

| `coef0`        | Influence of higher-order terms | High values emphasize high-degree terms           

| `C`            | Regularization strength         | Low = smoother margin, high = stricter fit to data |



SIMILARITY FEATURES :

instead of adding or mapping polynomial features another approach is to map the input points to some predefined points(landmarks) and see how similar they are 

see fig 5-8 :

left plot:
x axis = original 1d feature x1
y axis = similarity score computed using gaussian RBF 
two landmarks x2 = -2, x3 = 1 shown with red dots
the green and blue curves represent the similarity of x1 to x2 and x3
green curve point = (x1, similarity score with x2)
blue curve  = (x1, similarity score with x3)
this transforms a 1d point to 2d point 

right plot :
we have two landmarks x2 and x3, we will use the gaussian rbf formula(see copy 1)
calculated distance from x2 = 0.74
calculated distance from x3 = 0.30
we got a new vector (0.74, 0.30)
this means the original 1d input is now mapped to a 2d point based on its similarity to the two landmarks 
and as you can see in the plot the points are linearly separable now shown by the red line(decision boundary)

this transformations allows nonlinear relationships to become linear by using gaussian rbf to turn distance to similarities 
this is the basis of rbf kernel in svm which allows powerful non linear classification without computing thsese new features 

how to select landmarks:
use all training instances as landmark, but doing that will create many dimensions and thus increase the chances that the transformed training set will be linearly seperable now
the downside is if you have m instances and n features you will  end up with m instances and m features, if your training set is very large you end up with equally large no of features 

GAUSSIAN RBF KERNEL 

see copy 1
x is your local instance 
l is your landmark
y(gamma) controls the width of bell curve, low y wide smooth curve model is simple, high y narrow spiky curves model becomes sensitive to individual points 

the kernel trick :
without the kernel trick you have to manually calculate the similarity scores with respect to all landmarks and build a new feature vector for each instance which is computationally expensive 
but using the kernel trick svm can act as if it mapped inouts to a high dimensional similarity space without actually doing the transformations 

lets see the code :

In [11]:
rbf_kernel_svm_clf = make_pipeline(
    StandardScaler(),
    SVC(kernel="rbf", gamma=5, C=0.001)
)

rbf_kernel_svm_clf.fit(x,y)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc', SVC(C=0.001, gamma=5))])

the above model is represented by fig 5-9

the sharper the border between colors = more complex boundary 

top left :
low gamma = wide rbf kernel, smooth decision boundary
low C = tolerant of errors maximizes margin
the model is underfitting the decision boundary is almost linear missing the data pattern

top right :
low gamma = still smooth boundary
high C = punishes missclassification heavily tries to get all points right 
model starts to bend the boundary to fit the data better - less underfitting 

bottom left : (our model)
high gamma = narrow rbf kernel, focuses on individual data points 
low C = still tolerant of misclassification
decision boundary is complex but still not overfitting due to low C, some misclassifications are allowed

bottom right :
high gamma = high sensitivity to training data
high C = tries to classify all training point correctly 
overfitting the model learns data too cloely, you can see a very tight irregular decision boundary hugging the points 


there are other kernels too, some kernels are specialized for specific data structures, string kernels are sometimes used when classifying text documents or dna sequences 
with so many kernels to decide one should always start with the linear kernel first the LinearSVC class is much faster than SVC(kernel="linear") especially in large training sets 
if its not too large you should try kernelazied SVMs starting with gaussian rbf or you can also try experimenting with other kernels using hyperparameter search and if there are kernels specialized for your training set data you should try that too 

SVM CLASSES AND COMPLEXITIES :

linear svc :
efficient for large datasets as long as the feature count is not large, no kernel tricks so no non linear transformation using rbf or polynomial
time complexity = O(m x n)
m  = no of training instances 
n = no of features
the algo takes longer time if you need higher precision which is called tol
smaller tol = higher precision
higher tol = lower precision
in most cases default tol works well